In [4]:
cd ~/src/Classes/Ansible/

/Users/rick446/src/Classes/Ansible


# Roles: _Really_ organizing your code

A *Role* is a directory structure organizing different parts of plays in ways that let them be included in other playbooks easily:

- **role_name**
  - files
  - templates
  - tasks
  - handlers
  - vars
  - defaults
  - meta

If you have a role defined, it cab be used in a play using the `roles:` key, e.g.:

```
---
- hosts: all
  roles:
    - my_custom_role
```

This means to look in `my_custom_role` files named `main.yml` and add them to the play. Particularly, 

- If `my_custom_role/tasks/main.yml` exists, it is added to the `tasks:` part of the play
- If `my_custom_role/handlers/main.yml` exists, it is added to the `handlers:` part of the play
- If `my_custom_role/vars/main.yml` exists, it is added to the `vars:` part of the play
- If `my_custom_role/defaults/main.yml` exists, it is added to the `vars:` part of the play (as super-low priority)
- If `my_custom_role/meta/main.yml` exists, it is added to the `roles:` part of the play (think "sub-roles")

# Ansible Galaxy: A shared repository of Roles

To use Ansible Galaxy, we first should set up a `roles_path` in `ansible.cfg`. Mine looks like this now:

```
[defaults]
# ...
roles_path = ./roles
```

Now we can browse the [Ansible Galaxy](https://galaxy.ansible.com/) website and use the `ansible-galaxy` command-line tool to install roles. 

## Examining a popular Galaxy role: nginx

In [5]:
%%bash
ansible-galaxy install bennojoy.nginx
tree roles

roles
└── bennojoy.nginx
    ├── README.md
    ├── defaults
    │   └── main.yml
    ├── files
    │   └── epel.repo
    ├── handlers
    │   └── main.yml
    ├── meta
    │   └── main.yml
    ├── tasks
    │   └── main.yml
    ├── templates
    │   ├── default.conf.j2
    │   ├── default.j2
    │   ├── nginx.conf.j2
    │   └── site.j2
    └── vars
        └── main.yml

8 directories, 11 files


 [WARNING]: - bennojoy.nginx (master) is already installed - use --force to
change version to unspecified


 To use the role, we just need to add it to the `roles` key in our playbook. In this case, we'll also add some role variables to it:

In [6]:
cat playbooks/setup-webservers.yaml

- hosts: web
  roles:
    - role: nginx
      nginx_http_params:
        sendfile: 'on'
        access_log: '/var/log/nginx/access.log'
      nginx_sites:
       - server:
          file_name: foo
          listen: 8080
          server_name: localhost
          root: '/tmp/site1'
          location1: {name: /, try_files: '$uri $uri/ /index.html'}
          location2: {name: /images/, try_files: '$uri $uri/ /index.html'}
       - server:
          file_name: bar
          listen: 9090
          server_name: ansible
          root: '/tmp/site2'
          location1: {name: /, try_files: '$uri $uri/ /index.html'}
          location2: {name: /images/, try_files: '$uri $uri/ /index.html'}

In [ ]:
%%bash
ansible-playbook playbooks/setup-webservers.yaml

Here is another playbook to set up per-user directories on the web hosts:

In [13]:
cat playbooks/make-site.yaml

- hosts: web

  pre_tasks:
  - name: Ensure public_html exists
    file: path=~/public_html state=directory mode=755
  - name: Put a file in public_html
    file: path=~/public_html/index.html state=touch mode=644
  - name: Put a message in the file
    lineinfile: line="Hello there {{ansible_ssh_user}}" dest=~/public_html/index.html

  roles:
  - role: bennojoy.nginx
    become: on
    nginx_sites:
    - server:
        file_name: "{{ansible_ssh_user}}-site"
        location1:
          name: "/"
          root: "/usr/share/nginx/html"
        location2:
          name: "/~{{ansible_ssh_user}}"
          alias: "/home/{{ansible_ssh_user}}/public_html"
          try_files: "$uri $uri/ /index.html"

